<a href="https://colab.research.google.com/github/Anil-matcha/langchain-tutorials/blob/main/Blog_to_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests langchain trafilatura openai BeautifulSoup

In [12]:
import requests
from bs4 import BeautifulSoup
wunder = requests.get("https://uuki.live/sitemap.xml")
parcala = BeautifulSoup(wunder.content, "xml")

urls = []

loc_tags = parcala.find_all('loc')

for loc in loc_tags:
    if 'learn/' in loc.get_text():
        urls.append(loc.get_text().strip())

print(urls)


['https://uuki.live/learn/10-best-ai-image-generators-in-2023', 'https://uuki.live/learn/10-best-discord-alternatives-for-better-experience', 'https://uuki.live/learn/5-best-online-community-platforms-for-2023', 'https://uuki.live/learn/5-tips-for-building-a-stronger-brand-community', 'https://uuki.live/learn/6-steps-to-hack-your-way-to-a-great-producthunt-launch', 'https://uuki.live/learn/7-best-solana-nft-marketplaces', 'https://uuki.live/learn/7-examples-of-captivating-community-welcome-emails', 'https://uuki.live/learn/7-tips-for-online-community-moderators', 'https://uuki.live/learn/8-tips-for-the-perfect-community-newsletter', 'https://uuki.live/learn/autogpt-vs-babyagi', 'https://uuki.live/learn/best-discord-servers-for-web3-developers', 'https://uuki.live/learn/community-app-the-text-messaging-app-for-creators-and-influencers', 'https://uuki.live/learn/create-your-own-social-network-like-facebook', 'https://uuki.live/learn/creative-membership-level-names-how-to-name-your-member

In [19]:
from langchain.tools import BaseTool
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import Field
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain, BaseCombineDocumentsChain
from langchain.chat_models import ChatOpenAI
import os, asyncio, trafilatura
from langchain.docstore.document import Document

def _get_text_splitter():
    return RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size = 500,
        chunk_overlap  = 20,
        length_function = len,
    )

class WebpageQATool(BaseTool):
    name = "query_webpage"
    description = "Browse a webpage and retrieve the information relevant to the question."
    text_splitter: RecursiveCharacterTextSplitter = Field(default_factory=_get_text_splitter)
    qa_chain: BaseCombineDocumentsChain

    def _run(self, question: str) -> str:
        docs = []
        for url in urls[:2]:
            result = trafilatura.extract(trafilatura.fetch_url(url))
            docs.append(Document(page_content=result, metadata={"source": url}))
        #docs = [Document(page_content=result, metadata={"source": url})]
        web_docs = self.text_splitter.split_documents(docs)
        results = []
        for i in range(0, len(web_docs), 4):
            input_docs = web_docs[i:i+4]
            window_result = self.qa_chain({"input_documents": input_docs, "question": question}, return_only_outputs=True)
            results.append(f"Response from window {i} - {window_result}")
        results_docs = [Document(page_content="\n".join(results), metadata={"source": url})]
        return self.qa_chain({"input_documents": results_docs, "question": question}, return_only_outputs=True)

    async def _arun(self, url: str, question: str) -> str:
        raise NotImplementedError

llm = ChatOpenAI(temperature=1.0, openai_api_key="your-openai-key")
query_website_tool = WebpageQATool(qa_chain=load_qa_with_sources_chain(llm))
print(query_website_tool.run("What are the best AI image generators ?"))

{'output_text': 'According to a list compiled by Uuki.live, the best AI image generators include DALL-E 2, MidJourney, and WOMBO Dream.\nSOURCES: https://uuki.live/learn/10-best-ai-image-generators-in-2023'}
